In [3]:
import pandas as pd
import wandb
api = wandb.Api()

groups = [
"CNN_DATA_ANALYSIS_reto_grande_garage_exponential",
"CNN_DATA_ANALYSIS_reto_grande_indoor_exponential",
"CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential",
"CNN_DATA_ANALYSIS_reto_grande_garage_raw",
"CNN_DATA_ANALYSIS_reto_grande_indoor_raw",
"CNN_DATA_ANALYSIS_reto_grande_outdoor_raw",
"CNN_DATA_ANALYSIS_reto_grande_garage_meters",
"CNN_DATA_ANALYSIS_reto_grande_indoor_meters",
"CNN_DATA_ANALYSIS_reto_grande_outdoor_meters",
"CNN_DATA_ANALYSIS_all_collections_garage_outdoor",
"CNN_DATA_ANALYSIS_all_collections_indoor_garage",
"CNN_DATA_ANALYSIS_all_collections_indoor_outdoor",
"CNN_DATA_ANALYSIS_all_spaces_reto_grande",
]

databases = ["raw","meters","exponential","all_collections","all_spaces"]
spaces = ["indoor","garage","outdoor"]

model_dict:dict = {}

# Project is specified by <entity/project-name>
for group in groups:
    runs = api.runs(
    "santi-mbenites-university-of-lisbon/wifi-rssi-gradient-search",
    filters={"group":group}
    )
    for run in runs:
        model_dict[group] = model_dict.get(group,[])
        model_dict[group].append(run.name)


In [4]:
display(model_dict)

{'CNN_DATA_ANALYSIS_reto_grande_garage_exponential': ['reto_grande_garage_exponential_run0_depth3_model4',
  'reto_grande_garage_exponential_run1_depth4_model5',
  'reto_grande_garage_exponential_run2_depth4_model2'],
 'CNN_DATA_ANALYSIS_reto_grande_indoor_exponential': ['reto_grande_indoor_exponential_run0_depth0_model3',
  'reto_grande_indoor_exponential_run1_depth4_model5',
  'reto_grande_indoor_exponential_run2_depth1_model7'],
 'CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential': ['reto_grande_outdoor_exponential_run0_depth1_model2',
  'reto_grande_outdoor_exponential_run1_depth3_model2',
  'reto_grande_outdoor_exponential_run2_depth3_model4'],
 'CNN_DATA_ANALYSIS_reto_grande_garage_raw': ['reto_grande_garage_raw_run0_depth2_model6',
  'reto_grande_garage_raw_run1_depth3_model1',
  'reto_grande_garage_raw_run2_depth2_model4'],
 'CNN_DATA_ANALYSIS_reto_grande_indoor_raw': ['reto_grande_indoor_raw_run0_depth1_model3',
  'reto_grande_indoor_raw_run1_depth4_model2',
  'reto_grande_ind

In [5]:
import os
from model_evaluation import evaluate_model_on_collections
from consts import (
    cnn_group_data, BATCHSIZE, all_collections
)
from feature_lists import DATASET_TO_FEATURE

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

results = {
    "CNN_DATA_ANALYSIS_reto_grande_garage_exponential" : [],
    "CNN_DATA_ANALYSIS_reto_grande_indoor_exponential" : [],
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential" : [],
    "CNN_DATA_ANALYSIS_reto_grande_garage_raw" : [],
    "CNN_DATA_ANALYSIS_reto_grande_indoor_raw" : [],
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_raw" : [],
    "CNN_DATA_ANALYSIS_reto_grande_garage_meters" : [],
    "CNN_DATA_ANALYSIS_reto_grande_indoor_meters" : [],
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_meters" : [],
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor" : [],
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage" : [],
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor" : [],
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande" : [],
}
for current_group,all_runs in model_dict.items():

    for current_run in all_runs:
        current_group_data      = cnn_group_data[current_group]
        current_model_storage   = current_group_data["model_storage"]
        current_database        = current_group_data["database"]
        current_model_path      = f"{current_model_storage}/{current_run}.pt"
        feature_list            = DATASET_TO_FEATURE[current_database]
        relative_coeficient     = current_group_data["relative_coeficient"]
        model_output_name       = current_run
        val_collection          = current_group_data["validation_collections"]

        for collections_name, current_collections in val_collection.items(): 
            
            if not os.path.exists(current_model_path):
                print(f"Model doesnt exist {model_output_name} with path {current_model_path}")
                continue
            
            try:
                mae, mse, rmse, n = evaluate_model_on_collections(
                    current_model_path, current_collections, current_database, BATCHSIZE, None, feature_list
                )
            except Exception as e:
                print(f"Failed Processing {model_output_name} with path {current_model_path}")
                print(e)
                continue
            mae_final = mae * relative_coeficient
            mse_final = mse * relative_coeficient
            rmse_final = rmse * relative_coeficient
            print(f"{model_output_name} | Collections : {collections_name} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
            results[current_group].append((model_output_name, collections_name, mae_final, mse_final, rmse_final))


reto_grande_garage_exponential_run0_depth3_model4 | Collections : reto_grande_outdoor | N=19910 | MAE=7.969518 | MSE=3.459033 | RMSE=10.520887
reto_grande_garage_exponential_run0_depth3_model4 | Collections : reto_grande_indoor | N=12243 | MAE=10.536154 | MSE=5.120799 | RMSE=12.800999
reto_grande_garage_exponential_run0_depth3_model4 | Collections : reto_grande_garage | N=23160 | MAE=4.104213 | MSE=0.922476 | RMSE=5.433162
reto_grande_garage_exponential_run0_depth3_model4 | Collections : all_reto_grande | N=55313 | MAE=6.919228 | MSE=2.764836 | RMSE=9.406102
reto_grande_garage_exponential_run1_depth4_model5 | Collections : reto_grande_outdoor | N=19910 | MAE=7.954384 | MSE=3.473481 | RMSE=10.542836
reto_grande_garage_exponential_run1_depth4_model5 | Collections : reto_grande_indoor | N=12243 | MAE=10.556116 | MSE=5.162819 | RMSE=12.853413
reto_grande_garage_exponential_run1_depth4_model5 | Collections : reto_grande_garage | N=23160 | MAE=3.953435 | MSE=0.877636 | RMSE=5.299466
reto_gra

In [6]:
import pandas as pd
for group, rows in results.items():

    df = pd.DataFrame(rows, columns=["model","collections", "mae", "mse", "rmse"])
    print(f"GROUP : {group}")
    styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
    display(styled)

GROUP : CNN_DATA_ANALYSIS_reto_grande_garage_exponential


,model,collections,mae,mse,rmse
0,reto_grande_garage_exponential_run0_depth3_model4,reto_grande_outdoor,7.969518,3.459033,10.520887
1,reto_grande_garage_exponential_run0_depth3_model4,reto_grande_indoor,10.536154,5.120799,12.800999
2,reto_grande_garage_exponential_run0_depth3_model4,reto_grande_garage,4.104213,0.922476,5.433162
3,reto_grande_garage_exponential_run0_depth3_model4,all_reto_grande,6.919228,2.764836,9.406102
4,reto_grande_garage_exponential_run1_depth4_model5,reto_grande_outdoor,7.954384,3.473481,10.542836
5,reto_grande_garage_exponential_run1_depth4_model5,reto_grande_indoor,10.556116,5.162819,12.853413
6,reto_grande_garage_exponential_run1_depth4_model5,reto_grande_garage,3.953435,0.877636,5.299466
7,reto_grande_garage_exponential_run1_depth4_model5,all_reto_grande,6.855022,2.760498,9.398721
8,reto_grande_garage_exponential_run2_depth4_model2,reto_grande_outdoor,7.911217,3.397264,10.426527
9,reto_grande_garage_exponential_run2_depth4_model2,reto_grande_indoor,10.716594,5.278601,12.996739


GROUP : CNN_DATA_ANALYSIS_reto_grande_indoor_exponential


,model,collections,mae,mse,rmse
0,reto_grande_indoor_exponential_run0_depth0_model3,reto_grande_outdoor,12.927281,7.611803,15.606976
1,reto_grande_indoor_exponential_run0_depth0_model3,reto_grande_indoor,1.963650,0.246349,2.807698
2,reto_grande_indoor_exponential_run0_depth0_model3,reto_grande_garage,13.066334,8.083279,16.083064
3,reto_grande_indoor_exponential_run0_depth0_model3,all_reto_grande,10.558810,6.178942,14.061512
4,reto_grande_indoor_exponential_run1_depth4_model5,reto_grande_outdoor,12.935176,7.599653,15.594515
5,reto_grande_indoor_exponential_run1_depth4_model5,reto_grande_indoor,1.626562,0.196703,2.508884
6,reto_grande_indoor_exponential_run1_depth4_model5,reto_grande_garage,13.269352,8.144213,16.143569
7,reto_grande_indoor_exponential_run1_depth4_model5,all_reto_grande,10.572045,6.189093,14.073059
8,reto_grande_indoor_exponential_run2_depth1_model7,reto_grande_outdoor,12.855905,7.539936,15.533125
9,reto_grande_indoor_exponential_run2_depth1_model7,reto_grande_indoor,1.701109,0.207398,2.576187


GROUP : CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential


...

,model,collections,mae,mse,rmse
0,all_collections_indoor_garage_run0_depth1_model7,all_collections_garage,6.392071,2.056900,8.113002
1,all_collections_indoor_garage_run0_depth1_model7,all_collections_outdoor,9.262869,4.003126,11.318128
2,all_collections_indoor_garage_run0_depth1_model7,all_collections_indoor,9.425117,4.404973,11.872621
3,all_collections_indoor_garage_run0_depth1_model7,all_collections_garage_outdoor,8.352154,3.385717,10.408791
4,all_collections_indoor_garage_run0_depth1_model7,all_collections_garage_indoor,7.196984,2.680034,9.260729
5,all_collections_indoor_garage_run0_depth1_model7,all_collections_indoor_outdoor,9.286188,4.060881,11.399482
6,all_collections_indoor_garage_run0_depth1_model7,all_collections_garage_outdoor_indoor,8.462473,3.490514,10.568654
7,all_collections_indoor_garage_run1_depth2_model6,all_collections_garage,5.923733,1.834116,7.661052
8,all_collections_indoor_garage_run1_depth2_model6,all_collections_outdoor,9.396251,4.175633,11.559423
9,all_collections_indoor_garage_run1_depth2_model6,all_collections_indoor,6.071142,2.609023,9.137217


GROUP : CNN_DATA_ANALYSIS_all_collections_indoor_outdoor


,model,collections,mae,mse,rmse
0,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_garage,8.998065,3.945343,11.236146
1,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_outdoor,6.723179,2.147950,8.290620
2,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_indoor,8.301635,3.886636,11.152235
3,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_garage_outdoor,7.444851,2.718144,9.326339
4,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_garage_indoor,8.813246,3.929763,11.213939
5,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_indoor_outdoor,6.950041,2.397841,8.759618
6,all_collections_indoor_outdoor_run0_depth4_model2,all_collections_garage_outdoor_indoor,7.532943,2.838286,9.530223
7,all_collections_indoor_outdoor_run1_depth3_model4,all_collections_garage,9.170018,4.100232,11.454581
8,all_collections_indoor_outdoor_run1_depth3_model4,all_collections_outdoor,6.391999,1.972941,7.945698
9,all_collections_indoor_outdoor_run1_depth3_model4,all_collections_indoor,6.359613,2.896366,9.627238


GROUP : CNN_DATA_ANALYSIS_all_spaces_reto_grande


,model,collections,mae,mse,rmse
0,all_spaces_reto_grande_run0_depth4_model0,reto_grande_outdoor,5.397766,1.663568,7.296176
1,all_spaces_reto_grande_run0_depth4_model0,reto_grande_indoor,3.942647,1.088460,5.901754
2,all_spaces_reto_grande_run0_depth4_model0,reto_grande_garage,4.896820,1.268340,6.370782
3,all_spaces_reto_grande_run0_depth4_model0,all_reto_grande,4.865939,1.370788,6.623082
4,all_spaces_reto_grande_run1_depth2_model3,reto_grande_outdoor,5.194956,1.534096,7.006501
5,all_spaces_reto_grande_run1_depth2_model3,reto_grande_indoor,3.256929,0.860236,5.246670
6,all_spaces_reto_grande_run1_depth2_model3,reto_grande_garage,4.703547,1.183628,6.154357
7,all_spaces_reto_grande_run1_depth2_model3,all_reto_grande,4.560235,1.238200,6.294632
8,all_spaces_reto_grande_run2_depth2_model7,reto_grande_outdoor,5.608169,1.761677,7.508240
9,all_spaces_reto_grande_run2_depth2_model7,reto_grande_indoor,4.084444,1.066286,5.841332


In [7]:
import re
import pandas as pd
import matplotlib.pyplot as plt


combined_rows = []

for group, rows in results.items():
    
    dataset = max((d for d in databases if d in group), key=len, default=None)
    space = max((s for s in spaces if s in group), key=len, default=None)
    
    for current_row in rows:
        print(current_row)
        (model_output_name, collections_name, mae_final, mse_final, rmse_final) = current_row
        combined_rows.append((group,collections_name,mae_final,mse_final,rmse_final))

print(combined_rows)

('reto_grande_garage_exponential_run0_depth3_model4', 'reto_grande_outdoor', 7.969518410890082, 3.459033144592101, 10.520886874543763)
('reto_grande_garage_exponential_run0_depth3_model4', 'reto_grande_indoor', 10.536154204856762, 5.120799380658178, 12.800999186823725)
('reto_grande_garage_exponential_run0_depth3_model4', 'reto_grande_garage', 4.104213284046127, 0.9224764850679045, 5.433161834712172)
('reto_grande_garage_exponential_run0_depth3_model4', 'all_reto_grande', 6.9192279298396935, 2.7648359826069178, 9.406101819745594)
('reto_grande_garage_exponential_run1_depth4_model5', 'reto_grande_outdoor', 7.954384267587629, 3.473481070893545, 10.542836158671605)
('reto_grande_garage_exponential_run1_depth4_model5', 'reto_grande_indoor', 10.556116164105493, 5.162819207576349, 12.853412567969768)
('reto_grande_garage_exponential_run1_depth4_model5', 'reto_grande_garage', 3.9534345328704887, 0.8776356641904471, 5.2994661291581355)
('reto_grande_garage_exponential_run1_depth4_model5', 'all

In [8]:
import pandas as pd

singles = {
    "CNN_DATA_ANALYSIS_reto_grande_garage_exponential" : "reto_grande_garage_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_exponential" : "reto_grande_indoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential" : "reto_grande_outdoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_garage_raw" : "reto_grande_garage_raw",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_raw" : "reto_grande_indoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_raw" : "reto_grande_outdoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_garage_meters" : "reto_grande_garage_meters",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_meters" : "reto_grande_indoor_meters",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_meters" : "reto_grande_outdoor_meters",
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande" : "all_spaces_reto_grande"
}

spaces = {
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor" : "all_collections_garage_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage" : "all_collections_indoor_garage",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor" : "all_collections_indoor_outdoor"
}

spaces_datasets = []
singles_datasets = []

for row in combined_rows:
    (group,collections_name,mae_final,mse_final,rmse_final) = row
    if group in spaces:
        group_defnition = spaces[group]
        spaces_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    elif group in singles:
        group_defnition = singles[group]
        singles_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    else:
        print(f"ERROR : {group}")

print(spaces_datasets)
print(singles_datasets)


[('all_collections_garage_outdoor', 'all_collections_garage', 7.1768893468050505, 2.423679925161545, 8.80668823140512), ('all_collections_garage_outdoor', 'all_collections_outdoor', 6.8546262546911505, 2.1447211116921907, 8.284387459199992), ('all_collections_garage_outdoor', 'all_collections_indoor', 13.377179380997159, 8.34443757410919, 16.34080788613262), ('all_collections_garage_outdoor', 'all_collections_garage_outdoor', 6.956858899655849, 2.2332161230838383, 8.453574151723211), ('all_collections_garage_outdoor', 'all_collections_garage_indoor', 8.822325022369691, 3.9949322870866486, 11.306539399249125), ('all_collections_garage_outdoor', 'all_collections_indoor_outdoor', 7.792072177051306, 3.035767330221771, 9.856193715988779), ('all_collections_garage_outdoor', 'all_collections_garage_outdoor_indoor', 7.616980410742051, 2.86155658149579, 9.569211598029655), ('all_collections_garage_outdoor', 'all_collections_garage', 7.179323705140088, 2.423619647365269, 8.806578717963555), ('al

In [9]:
import pandas as pd

# rows: list[tuple] like (dataset, collections, mae, mse, rmse)
def make_wide_by_collection(rows, *,
                            caption="Modelos – erros médios por coleção",
                            label="tab:model_metrics_by_collection"):
    # Build DataFrame
    df = pd.DataFrame(rows, columns=["dataset", "collections", "mae", "mse", "rmse"])

    if df.empty:
        print("No data provided.")
        return None, None, None

    # Robust pivot (handles duplicate (dataset, collections) by averaging)
    df_wide = (
        df.pivot_table(index="dataset",
                       columns="collections",
                       values=["mae", "mse", "rmse"],
                       aggfunc="mean")
          .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
          .sort_index(axis=1, level=[0, 1])        # nice column order
          .sort_index(axis=0)                      # sort datasets
    )

    # Optional: round displayed values only
    styled = (
        df_wide.style
              .format(precision=2, decimal=",")
              .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=caption,
        label=label,
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
    )

    return df, df_wide, (styled, latex.replace("_", "\\_"))


def latex_with_lines(df, *args, **kwargs):
    kwargs['column_format'] = '|'.join([''] + ['l'] * df.index.nlevels
                                            + ['r'] * df.shape[1] + [''])
    res = df.to_latex(*args, **kwargs)
    return res.replace('\\\\\n', '\\\\ \\midrule\n')

In [10]:

# Build DataFrame
df = pd.DataFrame(spaces_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])

# Robust pivot (handles duplicate (dataset, collections) by averaging)
df_wide = (
    df.pivot_table(index="dataset",
                    columns="collections",
                    values=["mae", "mse", "rmse"],
                    aggfunc="mean")
        .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
        .sort_index(axis=1, level=[0, 1])        # nice column order
        .sort_index(axis=0)                      # sort datasets
)

df_wide = df_wide.transpose()

# Optional: round displayed values only
styled = (
    df_wide.style
            .format(precision=2, decimal=",")
            .background_gradient(cmap="RdYlGn_r")
)

latex = styled.to_latex(
    caption="Dataset Combination errors",
    label="tab:model_metrics_by_collection_spaces",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
)


display(styled)
print(latex.replace("_", "\\_"))




\begin{table}[H]
\caption{Dataset Combination errors}
\label{tab:model\_metrics\_by\_collection\_spaces}
\begin{tabular}{llrrr}
 & dataset & all\_collections\_garage\_outdoor & all\_collections\_indoor\_garage & all\_collections\_indoor\_outdoor \\
collections &  &  &  &  \\
\multirow[c]{3}{*}{all\_collections\_garage} & mae & {\cellcolor[HTML]{C3E67D}} \color[HTML]{000000} 7,17 & {\cellcolor[HTML]{E9F6A1}} \color[HTML]{000000} 6,18 & {\cellcolor[HTML]{F88950}} \color[HTML]{F1F1F1} 9,03 \\
 & mse & {\cellcolor[HTML]{04703B}} \color[HTML]{F1F1F1} 2,42 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,97 & {\cellcolor[HTML]{69BE63}} \color[HTML]{F1F1F1} 3,97 \\
 & rmse & {\cellcolor[HTML]{F4FAB0}} \color[HTML]{000000} 8,79 & {\cellcolor[HTML]{FED27F}} \color[HTML]{000000} 7,95 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 11,27 \\
\multirow[c]{3}{*}{all\_collections\_garage\_indoor} & mae & {\cellcolor[HTML]{F4FAB0}} \color[HTML]{000000} 8,80 & {\cellcolor[HTML]{F8FCB6}} \color[HTM

In [15]:



# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])

# Robust pivot (handles duplicate (dataset, collections) by averaging)
df_wide = (
    df.pivot_table(index="dataset",
                    columns="collections",
                    values=["mae", "mse", "rmse"],
                    aggfunc="mean")
        .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
        .sort_index(axis=1, level=[0, 1])        # nice column order
        .sort_index(axis=0)                      # sort datasets
)

datasets = ["all_reto_grande",	"reto_grande_garage",	"reto_grande_indoor",	"reto_grande_outdoor"]

for dataset in datasets:
    df_current = df_wide[dataset]
    df_current = df_current.sort_values('dataset')
    # Optional: round displayed values only
    styled = (
        df_current.style
                .format(precision=2, decimal=",")
                .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=f"{dataset} error",
        label=f"tab:cnn_all_datasets_{dataset}",
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
        hrules=True,
    )
    

    display(styled)
    print(latex.replace("_", "\\_"))




,mae,mse,rmse
dataset,,,
all_spaces_reto_grande,"4,81","1,34","6,56"
reto_grande_garage_exponential,"6,89","2,77","9,41"
reto_grande_garage_meters,"6,31","78,94","8,88"
reto_grande_garage_raw,"6,22","23,90","8,74"
reto_grande_indoor_exponential,"10,54","6,15","14,03"
reto_grande_indoor_meters,"10,28","189,71","13,77"
reto_grande_indoor_raw,"10,06","57,33","13,54"
reto_grande_outdoor_exponential,"8,25","3,77","10,99"
reto_grande_outdoor_meters,"8,36","129,39","11,37"


\begin{table}[H]
\caption{all\_reto\_grande error}
\label{tab:cnn\_all\_datasets\_all\_reto\_grande}
\begin{tabular}{lrrr}
\toprule
 & mae & mse & rmse \\
dataset &  &  &  \\
\midrule
all\_spaces\_reto\_grande & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 4,81 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,34 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 6,56 \\
reto\_grande\_garage\_exponential & {\cellcolor[HTML]{C5E67E}} \color[HTML]{000000} 6,89 & {\cellcolor[HTML]{016A38}} \color[HTML]{F1F1F1} 2,77 & {\cellcolor[HTML]{CFEB85}} \color[HTML]{000000} 9,41 \\
reto\_grande\_garage\_meters & {\cellcolor[HTML]{8CCD67}} \color[HTML]{000000} 6,31 & {\cellcolor[HTML]{DDF191}} \color[HTML]{000000} 78,94 & {\cellcolor[HTML]{ABDB6D}} \color[HTML]{000000} 8,88 \\
reto\_grande\_garage\_raw & {\cellcolor[HTML]{84CA66}} \color[HTML]{000000} 6,22 & {\cellcolor[HTML]{279F53}} \color[HTML]{F1F1F1} 23,90 & {\cellcolor[HTML]{A0D669}} \color[HTML]{000000} 8,74 \\
reto\_grande\_indoor\_expon

,mae,mse,rmse
dataset,,,
all_spaces_reto_grande,"4,86","1,26","6,34"
reto_grande_garage_exponential,"4,02","0,90","5,37"
reto_grande_garage_meters,"2,99","18,72","4,32"
reto_grande_garage_raw,"3,12","6,18","4,44"
reto_grande_indoor_exponential,"13,14","8,06","16,06"
reto_grande_indoor_meters,"12,76","246,95","15,71"
reto_grande_indoor_raw,"12,73","77,34","15,73"
reto_grande_outdoor_exponential,"8,04","3,37","10,38"
reto_grande_outdoor_meters,"8,92","130,45","11,42"


\begin{table}[H]
\caption{reto\_grande\_garage error}
\label{tab:cnn\_all\_datasets\_reto\_grande\_garage}
\begin{tabular}{lrrr}
\toprule
 & mae & mse & rmse \\
dataset &  &  &  \\
\midrule
all\_spaces\_reto\_grande & {\cellcolor[HTML]{5AB760}} \color[HTML]{F1F1F1} 4,86 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,26 & {\cellcolor[HTML]{51B35E}} \color[HTML]{F1F1F1} 6,34 \\
reto\_grande\_garage\_exponential & {\cellcolor[HTML]{1B9950}} \color[HTML]{F1F1F1} 4,02 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,90 & {\cellcolor[HTML]{16914D}} \color[HTML]{F1F1F1} 5,37 \\
reto\_grande\_garage\_meters & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 2,99 & {\cellcolor[HTML]{128A49}} \color[HTML]{F1F1F1} 18,72 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 4,32 \\
reto\_grande\_garage\_raw & {\cellcolor[HTML]{036E3A}} \color[HTML]{F1F1F1} 3,12 & {\cellcolor[HTML]{05713C}} \color[HTML]{F1F1F1} 6,18 & {\cellcolor[HTML]{026C39}} \color[HTML]{F1F1F1} 4,44 \\
reto\_grande\_indoor\_

,mae,mse,rmse
dataset,,,
all_spaces_reto_grande,"3,76","1,00","5,66"
reto_grande_garage_exponential,"10,60","5,19","12,88"
reto_grande_garage_meters,"9,47","132,75","11,52"
reto_grande_garage_raw,"9,48","40,98","11,45"
reto_grande_indoor_exponential,"1,76","0,22","2,63"
reto_grande_indoor_meters,"1,47","5,06","2,25"
reto_grande_indoor_raw,"1,61","1,72","2,35"
reto_grande_outdoor_exponential,"15,17","8,98","16,95"
reto_grande_outdoor_meters,"15,48","301,96","17,36"


\begin{table}[H]
\caption{reto\_grande\_indoor error}
\label{tab:cnn\_all\_datasets\_reto\_grande\_indoor}
\begin{tabular}{lrrr}
\toprule
 & mae & mse & rmse \\
dataset &  &  &  \\
\midrule
all\_spaces\_reto\_grande & {\cellcolor[HTML]{48AE5C}} \color[HTML]{F1F1F1} 3,76 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,00 & {\cellcolor[HTML]{75C465}} \color[HTML]{000000} 5,66 \\
reto\_grande\_garage\_exponential & {\cellcolor[HTML]{FECA79}} \color[HTML]{000000} 10,60 & {\cellcolor[HTML]{04703B}} \color[HTML]{F1F1F1} 5,19 & {\cellcolor[HTML]{FDAD60}} \color[HTML]{000000} 12,88 \\
reto\_grande\_garage\_meters & {\cellcolor[HTML]{FEEB9D}} \color[HTML]{000000} 9,47 & {\cellcolor[HTML]{E8F59F}} \color[HTML]{000000} 132,75 & {\cellcolor[HTML]{FEDA86}} \color[HTML]{000000} 11,52 \\
reto\_grande\_garage\_raw & {\cellcolor[HTML]{FEEB9D}} \color[HTML]{000000} 9,48 & {\cellcolor[HTML]{33A456}} \color[HTML]{F1F1F1} 40,98 & {\cellcolor[HTML]{FEDC88}} \color[HTML]{000000} 11,45 \\
reto\_grande\_in

,mae,mse,rmse
dataset,,,
all_spaces_reto_grande,"5,40","1,65","7,27"
reto_grande_garage_exponential,"7,95","3,44","10,50"
reto_grande_garage_meters,"8,23","115,89","10,76"
reto_grande_garage_raw,"7,83","34,00","10,43"
reto_grande_indoor_exponential,"12,91","7,58","15,58"
reto_grande_indoor_meters,"12,80","236,68","15,38"
reto_grande_indoor_raw,"12,14","68,24","14,78"
reto_grande_outdoor_exponential,"4,23","1,04","5,77"
reto_grande_outdoor_meters,"3,31","22,03","4,69"


\begin{table}[H]
\caption{reto\_grande\_outdoor error}
\label{tab:cnn\_all\_datasets\_reto\_grande\_outdoor}
\begin{tabular}{lrrr}
\toprule
 & mae & mse & rmse \\
dataset &  &  &  \\
\midrule
all\_spaces\_reto\_grande & {\cellcolor[HTML]{70C164}} \color[HTML]{000000} 5,40 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,65 & {\cellcolor[HTML]{7DC765}} \color[HTML]{000000} 7,27 \\
reto\_grande\_garage\_exponential & {\cellcolor[HTML]{F8FCB6}} \color[HTML]{000000} 7,95 & {\cellcolor[HTML]{026C39}} \color[HTML]{F1F1F1} 3,44 & {\cellcolor[HTML]{FFF5AE}} \color[HTML]{000000} 10,50 \\
reto\_grande\_garage\_meters & {\cellcolor[HTML]{FFFBB8}} \color[HTML]{000000} 8,23 & {\cellcolor[HTML]{FAFDB8}} \color[HTML]{000000} 115,89 & {\cellcolor[HTML]{FEEDA1}} \color[HTML]{000000} 10,76 \\
reto\_grande\_garage\_raw & {\cellcolor[HTML]{F4FAB0}} \color[HTML]{000000} 7,83 & {\cellcolor[HTML]{36A657}} \color[HTML]{F1F1F1} 34,00 & {\cellcolor[HTML]{FFF7B2}} \color[HTML]{000000} 10,43 \\
reto\_grande\_i